## Notebook Setup

### Imports
* **Run this cell immediately after opening the notebook by placing the cursor in the cell and hitting `SHIFT + ENTER`.**
* Otherwise, leave this cell alone

In [4]:
# Imports for telling Python where to find IRScout Python modules
import sys
import os
import re
sys.path.append('..')
    
# Set Current working directory to irsScouting2017\\Server
ptn = re.compile("jupyter$")
if ptn.search(os.getcwd()) is not None:
    os.chdir("..")
os.getcwd()

# IRScout Python Modules
import Server.scouting.output as out
from Server.scouting.event import EventDal

# Use the Bokeh plotting package
import holoviews as hv

# Other useful modules
import pandas as pd
import sqlalchemy as sa

# Get a database connection
connection_string = ('postgresql://irs1318:irs1318'
                     '@localhost:5432/scouting')
engine = sa.create_engine(connection_string)
conn = engine.connect()

In [5]:
os.getcwd()

'C:\\Users\\stacy\\OneDrive\\Projects\\scouting\\scouting2017\\irsScouting2017\\Server'

### Set Defaults
### Set Event
* **Next, set `current_event` to the applicable FIRST API event code and run this cell.**

In [6]:
# Set Chart Size
# bc.defaults.height = 300
# bc.defaults.width = 1200

# Set Current Event
current_event = "turing"
EventDal.set_current_event(current_event)
EventDal.get_current_event()

'turing'

### Export Excel Ranking File
* Run this cell to export the rankings spreadsheet.

In [7]:
fpath = out.get_Path("Report")
out.get_report(fpath)

### Get a Dataframe Containing all Measures

In [8]:
# Get Measures Dataframe
sql_measures = sa.text(
    "SELECT events.name AS event, matches.name AS match, levels.name AS level, dates.name AS date, "
    "teams.name AS team, alliances.name AS alliance, "
    "stations.name AS station, phases.name AS phase, actors.name AS actor, tasks.name AS task, "
    "measures.capability AS capability, measures.successes as successes, measures.attempts AS attempts, "
    "measures.cycle_times as cycle_times, measuretypes.name AS measuretype, reasons.name AS reason, "
    "attempts.name AS attempt "
    "FROM ((((((((((((measures FULL OUTER JOIN teams ON measures.team_id=teams.id) "
    "LEFT JOIN events ON measures.event_id=events.id) "
    "LEFT JOIN matches ON measures.match_id=matches.id) "
    "LEFT JOIN levels ON measures.level_id=levels.id) "
    "LEFT JOIN dates ON measures.date_id=dates.id) "
    "LEFT JOIN alliances ON measures.alliance_id=alliances.id) "
    "LEFT JOIN stations ON measures.station_id=stations.id) "
    "LEFT JOIN phases ON measures.phase_id=phases.id) "
    "LEFT JOIN actors ON measures.actor_id=actors.id) "
    "LEFT JOIN tasks ON measures.task_id=tasks.id) "
    "LEFT JOIN reasons ON measures.reason_id=reasons.id) "
    "LEFT JOIN measuretypes ON measures.measuretype_id=measuretypes.id) "
    "LEFT JOIN attempts ON measures.attempt_id=attempts.id "
    "WHERE events.name='" + current_event + "';")
measures = pd.read_sql(sql_measures, conn)
measures.query("task == 'shootHighBoiler'").query("phase == 'auto' or phase == 'teleop'")
#measures.query("task == 'climbRope' and phase == 'finish'")"

,event,match,level,date,team,alliance,station,phase,actor,task,capability,successes,attempts,cycle_times,measuretype,reason,attempt
9,turing,002-q,qual,2017-04-20T08:38:00,2990,blue,3,auto,robot,shootHighBoiler,0,0,0,0,count,na,summary
84,turing,005-q,qual,2017-04-20T09:02:00,1165,red,3,auto,robot,shootHighBoiler,0,1,1,0,count,na,summary
128,turing,004-q,qual,2017-04-20T08:54:00,108,blue,2,teleop,robot,shootHighBoiler,0,1,10,0,count,na,summary
217,turing,006-q,qual,2017-04-20T09:10:00,1595,blue,2,auto,robot,shootHighBoiler,0,1,1,0,count,na,summary
295,turing,008-q,qual,2017-04-20T09:26:00,3937,blue,1,teleop,robot,shootHighBoiler,0,0,4,0,count,na,summary
368,turing,009-q,qual,2017-04-20T09:34:00,2907,blue,3,auto,robot,shootHighBoiler,0,0,1,0,count,na,summary
382,turing,010-q,qual,2017-04-20T09:42:00,283,blue,3,auto,robot,shootHighBoiler,0,10,10,0,count,na,summary
399,turing,012-q,qual,2017-04-20T09:58:00,1595,red,3,auto,robot,shootHighBoiler,0,1,9,0,count,na,summary
413,turing,012-q,qual,2017-04-20T09:58:00,2102,red,2,auto,robot,shootHighBoiler,0,14,14,0,count,na,summary
448,turing,012-q,qual,2017-04-20T09:58:00,4586,blue,3,teleop,robot,shootHighBoiler,0,22,22,0,count,na,summary


### Get Number of Completed Matches"

In [9]:
# Set recent_matches to n to filter out all but the last n matches.
recent_matches= 12

select_recent_matches = sa.text(
    "with current AS (SELECT s.event, s.match, date from schedules sched, "
    "status s WHERE sched.event = s.event "
    "AND sched.match = s.match limit 1 ) "


    "SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event = c.event and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        recent_matches) + " ORDER by team, date desc")
df = pd.read_sql(select_recent_matches, conn)
num_matches = df[['team', 'level']].groupby('team').count()
num_matches.columns = ['matches']

### Create and Save Gears Chart
* This cell creates the gears chart and saves it to Server/web/data as a static, interactive html file.

**TODO:** Add an event code and timestamp to gears chart file name.

In [10]:
# Create summary data frame for placing gears
gears_df = measures.query("task == 'placeGear'").query("phase == 'auto' or phase == 'teleop'").groupby('team').sum()
gears_df['average'] = gears_df['successes'] / num_matches['matches']
gears_df = gears_df.sort_values('average', ascending = False)

gears_df

# Create gears chart in Bokeh
# gears_chart = bc.Bar(gears_df, values = 'average', label = bca.cat(sort = False), legend = None, color = 'purple',
#                      title = 'Average Gears Placed per Match', ylabel = "Gears Placed")
# bc.output_file('web/data/gears_chart.html', mode = 'inline')
# gears_df = measures.query("task == 'climbRope'").query("phase == 'finish'").groupby('team').sum()
# gears_df = gears_df.sort_values('successes', ascending = False)

# # Create gears chart in Bokeh
# gears_chart = bc.Bar(gears_df, values = 'successes', label = bca.cat(sort = False), legend = None, color = 'purple',
#                     title = 'Total Gears Placed', ylabel = "Gears Placed")
# bc.output_file('web/data/gears_chart.html')
# bc.show(gears_chart)

,capability,successes,attempts,cycle_times,average
team,,,,,
1296,0,47,50,0,4.700000
4206,0,47,54,0,4.700000
3737,0,44,46,0,4.400000
399,0,46,50,0,4.181818
3245,0,41,46,0,4.100000
3997,0,41,50,0,4.100000
4451,0,41,45,0,4.100000
4587,0,38,42,0,3.800000
5802,0,36,42,0,3.600000


In [11]:
# Create summary data frame for climbing rope
climbs_df = measures.query("task == 'climbRope'").query("phase == 'finish'").groupby('team').count()
climbs_df['average'] = climbs_df['successes'] / num_matches['matches']
climbs_df = climbs_df.sort_values('average', ascending = False)

# Create gears chart in Bokeh
climb_chart = bc.Bar(climbs_df, values = 'average', label = bca.cat(sort = False), legend = None, color = 'DarkGoldenRod',
                   title = 'Total Times Climbed', ylabel = "Times Climbed")
bc.output_file('web/data/climb_chart.html', mode = 'inline')
bc.show(climb_chart)

NameError: name 'bc' is not defined